# BERT Viz

In [2]:
#pgadmin
import os
import psycopg2 as pg


# For BERT
from transformers import DistilBertTokenizerFast, DistilBertModel

# For data manipulation and analysis
import pandas as pd
pd.options.display.max_colwidth = 200
import numpy as np
from sklearn.decomposition import PCA

# For interactive data visualization
import altair as alt


import re

# Data

In [22]:
repo_data = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/github_repos_157k.csv")

In [23]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adoptable JavaScript framework for building UI on the web.","<p align=""center""><a href=""https://vuejs.org"" rel=""noopener noreferrer"" target=""_blank""><img alt=""Vue logo"" src=""https://vuejs.org/images/logo.png"" width=""100""/></a></p>\n<p align=""center"">\n<a hr...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070,29611,185611,6250
1,facebook/react,"A declarative, efficient, and flexible JavaScript library for building user interfaces.",# [React](https://reactjs.org/) · [![GitHub license](https://img.shields.io/badge/license-MIT-blue.svg)](https://github.com/facebook/react/blob/main/LICENSE) [![npm version](https://img.shields.io...,JavaScript,"['javascript', 'react', 'frontend', 'declarative', 'ui', 'library']",12695,34352,171327,6718
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for Everyone,"<div align=""center"">\n<img src=""https://www.tensorflow.org/images/tf_logo_social.png""/>\n</div>\n\n[![Python](https://img.shields.io/pypi/pyversions/tensorflow.svg?style=plastic)](https://badge.fu...",C++,"['tensorflow', 'machine-learning', 'python', 'deep-learning', 'deep-neural-networks', 'neural-network', 'ml', 'distributed']",75671,84937,156754,8092
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript framework for developing responsive, mobile first projects on the web.","<p align=""center"">\n<a href=""https://getbootstrap.com/"">\n<img alt=""Bootstrap logo"" height=""165"" src=""https://getbootstrap.com/docs/5.0/assets/brand/bootstrap-logo-shadow.png"" width=""200""/>\n</a>\...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'scss', 'css-framework', 'sass']",19228,73981,151778,7079
4,ohmyzsh/ohmyzsh,"🙃 A delightful community-driven (with 1700+ contributors) framework for managing your zsh configuration. Includes nearly 300 optional plugins (rails, git, OSX, hub, capistrano, brew, ant, php, py...","<p align=""center""><img alt=""Oh My Zsh"" src=""https://s3.amazonaws.com/ohmyzsh/oh-my-zsh-logo.png""/></p>\n\nOh My Zsh is an open source, community-driven framework for managing your [zsh](https://ww...",Shell,"['shell', 'zsh-configuration', 'theme', 'terminal', 'productivity', 'hacktoberfest']",5447,22232,129314,2678
...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototype B for RFI # CDT-ADPQ-0117,"# CA Alerts, made with Very Little Gravitas for CDT RFI # CDT-ADPQ-0117\n\n[CA Alerts](https://alerts-ca.herokuapp.com) ([https://alerts-ca.herokuapp.com](https://alerts-ca.herokuapp.com)), submit...",CSS,"['prototype', 'messaging', 'emergency', 'govtech']",414,1,0,2
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414,0,0,1
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the application\r\n\r\nThis is the implementation of Library Management System (LMS). LMSs are used in libraries to track the different items in the library...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415,2,0,2
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for further information.,Animator\n========\n\nAnimator for hacklace. See hacklace.org for further information.\n\nInstallation steps\n==================\n\n1. Install and setup [maven](http://maven.apache.org/).\n2. Clon...,Java,[],415,0,0,0


In [24]:
# randomly select a subset
repo_data = repo_data.sample(2000)

In [25]:
# create a list of description text
python_description_text = repo_data["description"].tolist()


In [26]:
len(python_description_text)

2000

In [27]:
type(python_description_text[0])

str

# Encode/tokenize text data for BERT

In [28]:
from transformers import DistilBertTokenizerFast

In [29]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

The `tokenizer()` will break word tokens into word pieces, truncate to 312 tokens, and add padding and special BERT tokens.

In [30]:
tokenized_description = tokenizer(python_description_text, truncation=True, padding=True, return_tensors="pt")

In [57]:
python_description_text[7]

'Common source code used by MonTransit Android apps.'

In [56]:
' '.join(tokenized_description[7].tokens)

'[CLS] common source code used by mont ##ran ##sit android apps . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

## Load pre-trained BERT model

In [58]:
from transformers import DistilBertModel

In [35]:
#model = DistilBertModel.from_pretrained('distilbert-base-uncased').to("cuda")
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
# List of vocabulary word IDs for all the words in each document (aka each repo description)
doc_word_ids = []
# List of word vectors for all the words in each document (aka each repo description)
doc_word_vectors = []

# Below we will slice our repo description to ignore the first (0th) and last (-1) special BERT tokens
start_of_words = 1
end_of_words = -1

# Below we will index the 0th or first document, which will be the only document, since we're analzying one repo description at a time
first_document = 0

for i, description in enumerate(python_description_text):
  
    # Here we tokenize each poem with the DistilBERT Tokenizer
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True)

    # Here we extract the vocabulary word ids for all the words in the poem (the first or 0th document, since we only have one document)
    # We ignore the first and last special BERT tokens
    # We also convert from a Pytorch tensor to a numpy array
    doc_word_ids.append(inputs.input_ids[first_document].numpy()[start_of_words:end_of_words])

    # Here we send the tokenized poems to the GPU
    # The model is already on the GPU, but this poem isn't, so we send it to the GPU
    #inputs.to("cuda")
    
    # Here we run the tokenized poem through the DistilBERT model
    outputs = model(**inputs)

    # We take every element from the first or 0th document, from the 2nd to the 2nd to last position
    # Grabbing the last layer is one way of getting token vectors. There are different ways to get vectors with different pros and cons
    doc_word_vectors.append(outputs.last_hidden_state[first_document,start_of_words:end_of_words,:].detach().cpu().numpy())


In [ ]:
#Confirm that we have the same number of documents for both the tokens and the vectors:
len(doc_word_ids), len(doc_word_vectors)

In [ ]:
doc_word_ids[0], doc_word_vectors[0]

### Concatenate all word IDs/vectors for all doc

In [38]:
all_word_ids = np.concatenate(doc_word_ids)
all_word_vectors = np.concatenate(doc_word_vectors, axis=0)

In [39]:
# Calculating the length of each vector (Pythagorean theorem)
row_norms = np.sqrt(np.sum(all_word_vectors ** 2, axis=1))
# Dividing every vector by its length
all_word_vectors /= row_norms[:,np.newaxis]

## Find all word positions in a collection

In [43]:
def get_word_positions(words):
  
    """This function accepts a list of words, rather than a single word"""

    # Get word/vocabulary ID from BERT for each word
    word_ids = [tokenizer.vocab[word] for word in words]

    # Find all the positions where the words occur in the collection
    word_positions = np.where(np.isin(all_word_ids, word_ids))[0]

    return word_positions

In [44]:
get_word_positions(["python"])

array([   12,   215,   504,   707,   721,  1107,  1704,  2135,  2162,
        2187,  2446,  3496,  5657,  5945,  6609,  8765,  8833,  8930,
        9118,  9598,  9697, 10029, 10110, 10116, 10383, 10510, 11123,
       12531, 12989, 13292, 13361, 14381, 14565, 14747, 14867, 15121,
       15815, 16271, 16524, 17881, 17958, 19027, 20373, 20743, 21471,
       23056, 23887, 24366, 24392, 24967, 25111, 25297, 25335, 25973,
       27070, 27262, 27590, 28263, 28393, 28818, 29288, 29294, 29404,
       29407, 29474, 29555, 30304, 30574, 30648, 30741, 30748, 31073,
       31532, 31654, 32045, 35154, 35158, 35993, 36313, 36426, 37179,
       37199, 39013, 39257, 39292, 39425, 41444, 41889, 42258, 43252,
       43413, 43427, 43436, 43556, 44650, 44723, 44734, 45446, 45679,
       46372, 46514, 46942, 47310, 48745, 49880, 51481, 51593, 52164,
       52550, 52608, 52925, 53099, 53158, 53248, 54266, 56008, 57112,
       57917, 58551, 58715, 58998, 59657, 59672, 60601, 62693, 64363,
       64388, 65035,

In [45]:
word_positions = get_word_positions(["python"])

## Find word from word position

In [47]:
# Here we create an array so that we can go backwards from numeric token IDs to words
word_lookup = np.empty(tokenizer.vocab_size, dtype="O")

for word, index in tokenizer.vocab.items():
    word_lookup[index] = word

In [48]:
word_positions = get_word_positions(["python"])

for word_position in word_positions:
    print(word_position, word_lookup[all_word_ids[word_position]])

12 python
215 python
504 python
707 python
721 python
1107 python
1704 python
2135 python
2162 python
2187 python
2446 python
3496 python
5657 python
5945 python
6609 python
8765 python
8833 python
8930 python
9118 python
9598 python
9697 python
10029 python
10110 python
10116 python
10383 python
10510 python
11123 python
12531 python
12989 python
13292 python
13361 python
14381 python
14565 python
14747 python
14867 python
15121 python
15815 python
16271 python
16524 python
17881 python
17958 python
19027 python
20373 python
20743 python
21471 python
23056 python
23887 python
24366 python
24392 python
24967 python
25111 python
25297 python
25335 python
25973 python
27070 python
27262 python
27590 python
28263 python
28393 python
28818 python
29288 python
29294 python
29404 python
29407 python
29474 python
29555 python
30304 python
30574 python
30648 python
30741 python
30748 python
31073 python
31532 python
31654 python
32045 python
35154 python
35158 python
35993 python
36313 python


### We can also look for the 3 words that come before "python" and the 3 words that come after it.

In [50]:
word_positions = get_word_positions(["python"])

for word_position in word_positions:

    # Slice 3 words before "bank"
    start_pos = word_position - 3
    # Slice 3 words after "bank"
    end_pos = word_position + 4
    context_words = word_lookup[all_word_ids[start_pos:end_pos]]
    # Join the words together
    context_words = ' '.join(context_words)
    print(word_position, context_words)

12 ##logy data in python , together with
215 old repository a python api for fra
504 storm architecture the python sd ##k for
707 licensed , unofficial python client for the
721 agent written in python a humble project
1107 for bell media python tiny approx ##ima
1704 - virus checking python - su ##sea
2135 b ##x - python imported with h
2162 b ##x - python / with missing
2187 b ##x - python added [UNK] [UNK]
2446 ##cion ##al a python implementation of the
3496 server written in python a validation plug
5657 . org . python ##ic task management
5945 code generation . python ##ic wrap ##per
6609 ' s ecosystem python core package that
8765 ##ur ##ren ##cy python measurement platform for
8833 _ rep ##o python server utilities fork
8930 our api . python library for knowledge
9118 web server in python mo ##s ##n
9598 definitely not an python exploit . react
9697 64 - bit python ) quick ##fo
10029 all platforms . python 3 version of
10110 implementing binding for python ##2 . 7
10116 7 and al

In [54]:
def get_context(word_id, window_size=10):
  
    """Simply get the tokens that occur before and after word position"""

    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    context_words = " ".join(tokens)

    return context_words

In [61]:
def get_context_clean(word_id, window_size=10):
  
    """Get the tokens that occur before and after word position AND make them more readable"""

    keyword = word_lookup[all_word_ids[word_id]]
    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    # Make wordpieces slightly more readable
    # This is probably not the most efficient way to clean and correct for weird spacing
    context_words = " ".join(tokens)
    context_words = re.sub(r'\s+([##])', r'\1', context_words)
    context_words = re.sub(r'##', r'', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)
    context_words = re.sub('\s\'er', '\'er', context_words)
    context_words = re.sub(r'\s+([-,:?.!;])', r'\1', context_words)
    context_words = re.sub(r'([-\'"])\s+', r'\1', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)

    # Bold the keyword by putting asterisks around it
    if keyword in context_words:
        context_words = re.sub(f"\\b{keyword}\\b", f"**{keyword}**", context_words)
        context_words = re.sub(f"\\b({keyword}[esdtrlying]+)\\b", fr"**\1**", context_words)

    return context_words

## Viz

In [62]:
from IPython.display import Markdown, display

def print_md(string):
    display(Markdown(string))

In [63]:
word_positions = get_word_positions(["python"])

for word_position in word_positions:
    print_md(f"<br> {word_position}:  {get_context_clean(word_position)} <br>")

<br> 12:  a package for representing electrophysiology data in **python**, together with support for reading a wide range of <br>

<br> 215:  . distributed systems emulator--old repository a **python** api for fragalysis ( fragalysis. <br>

<br> 504:  . streaming processing system designed like apache storm architecture the **python** sdk for the rackspace cloud braintree drop <br>

<br> 707:  p2p framework an apache2 licensed, unofficial **python** client for the mygeotab api. linux <br>

<br> 721:  mygeotab api. linux agent written in **python** a humble project to tackle the hardest problem in society <br>

...

<br> 71950:  swift binary protocol client libraries a retro game engine for **python** a wordpress plugin to parse gutenberg <br>

<br> 72152:  a powerful spider ( web crawler ) system in **python**. a scratch pad repository to track efforts to produce <br>

<br> 72458:  qp data; pytrx is a **python** object-oriented programme created for the purpose of calculating <br>

<br> 72840:  library a c + + library ( bindings for **python** and c# ) for interfacing with the <br>

<br> 73857:  is is a high-performance fake data generator for **python**, which provides data for a variety of purposes in <br>

In [64]:
word_positions = get_word_positions(["python"])

keyword_contexts = []
keyword_contexts_tokens = []

for position in word_positions:
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

## PCA

In [66]:
from sklearn.decomposition import PCA

word_positions = get_word_positions(["python"])

pca = PCA(n_components=2)

pca.fit(all_word_vectors[word_positions,:].T)

PCA(n_components=2)

In [72]:
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens})
df.head()

,x,y,context,tokens
0,-0.082029,0.028445,"a package for representing electrophysiology data in **python**, together with support for reading a wide range of","a package for representing electro ##phy ##sio ##logy data in python , together with support for reading a wide range of"
1,-0.075349,-0.056566,. distributed systems emulator--old repository a **python** api for fragalysis ( fragalysis.,. distributed systems em ##ulator - - old repository a python api for fra ##gal ##ysis ( fra ##gal ##ysis .
2,-0.084960,0.013573,. streaming processing system designed like apache storm architecture the **python** sdk for the rackspace cloud braintree drop,. streaming processing system designed like apache storm architecture the python sd ##k for the racks ##pace cloud brain ##tree drop
3,-0.084530,0.008823,"p2p framework an apache2 licensed, unofficial **python** client for the mygeotab api. linux","p ##2 ##p framework an apache ##2 licensed , unofficial python client for the my ##ge ##ota ##b api . linux"
4,-0.081456,0.007989,mygeotab api. linux agent written in **python** a humble project to tackle the hardest problem in society,my ##ge ##ota ##b api . linux agent written in python a humble project to tackle the hardest problem in society


### Merge with original data

## Plot embeddings

In [68]:
import altair as alt

In [73]:
alt.Chart(df, title="Word Similarity: Python").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    # If you click a point, take you to the URL link 
    #href="link",
    # The categories that show up in the hover tooltip
    tooltip=[ 'context']
    ).interactive().properties(
    width=500,
    height=500
)

alt.Chart(...)

# Plot word embeddings from keywords (all at once)

In [79]:
# Tokenize all the poems
tokenized_repos = tokenizer(python_description_text_clean, truncation=True, padding=True, return_tensors="pt")

# Get a list of all the tokens for each poem
all_tokenized_repos = []
for i in range(len(tokenized_repos['input_ids'])):
    all_tokenized_repos.append(' '.join(tokenized_repos[i].tokens))

# Add them to the original DataFrame
repo_data['tokens'] = all_tokenized_repos

In [80]:
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers,tokens
21349,NeuralEnsemble/python-neo,"Neo is a package for representing electrophysiology data in Python, together with support for reading a wide range of neurophysiology file formats","===\nNeo\n===\n\nNeo is a Python package for working with electrophysiology data in Python, together\nwith support for reading a wide range of neurophysiology file formats, including\nSpike2, Neur...",Python,['hacktoberfest'],2533,178,187,33,"[CLS] neo is a package for representing electro ##phy ##sio ##logy data in python , together with support for reading a wide range of ne ##uro ##phy ##sio ##logy file formats [SEP] [PAD] [PAD] [PA..."
69869,kuzzleio/sdk-android,Official android SDK for Kuzzle,[![Build Status](https://api.travis-ci.org/kuzzleio/sdk-android.svg?branch=master)](https://travis-ci.org/kuzzleio/sdk-android) [![codecov.io](http://codecov.io/github/kuzzleio/sdk-android/coverag...,Java,"['kuzzle-sdk', 'android', 'kuzzle', 'swig', 'offline-first']",641,2,5,14,[CLS] official android sd ##k for ku ##zzle [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P...
6609,YunYang1994/TensorFlow2.0-Examples,"🙄 Difficult algorithm, Simple code.","<h2 align=""center""><code>🎉TensorFlow2.0-Examples🎉!</code></h2>\n<p align=""center"">""<i>Talk is cheap, show me the code.</i>"" \n ----- Linus Torvalds</p>\n<p align=""center"">\n<a href=""https://githu...",Jupyter Notebook,"['tensorflow2', 'tensorflow-examples', 'deep-learning', 'deep-neural-networks', 'machine-learning', 'gan', 'linear-regression', 'resnet', 'reinforcement-learning', 'image-classification', 'object-...",363,641,1494,60,"[CLS] [UNK] difficult algorithm , simple code . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD..."
100793,doruvil/TwitterSearchTool,This is a part of a functional iPhone app where the user can search for tweets on Twitter and see any particular tweet in a UIWebView while injecting it’s word count after the tweet.,Twitter Search Tool iOS\n=================\n\nThis is a part of a functional iPhone app where the user can search for tweets on Twitter and see any particular tweet in a UIWebView while injecting ...,Swift,[],796,0,1,1,[CLS] this is a part of a functional iphone app where the user can search for t ##wee ##ts on twitter and see any particular t ##wee ##t in a ui ##we ##b ##view while in ##ject ##ing it ’ s word c...
156163,genestudio/enterprise,Enterprise-grade component library for the Infor Design System,"# :tada: Soho XI Has Moved to Github! :tada:\n\nStarting at the end of release `4.6.0`, we moved the Soho XI codebase to Github! What this means is now anyone can view and contribute to the codeba...",HTML,[],10562,0,0,0,[CLS] enterprise - grade component library for the info ##r design system [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P...
...,...,...,...,...,...,...,...,...,...,...
132340,hlb9071/deeptest,开源优测-积微速成计划,## 开源优测-积微速成计划\n\n在这里大家组团相互督促学习，每日提交学习记录，完成每日任务\n\n在专用微信群里讨论、交流技术，拒绝打酱油，拒绝水...\n\n由苦叶子发起的一项免费的成长计划，从公众号：开源优测 的关注者每次筛选一批人员，一起成长\n\n包括但不限于以下方面的成长：\n\nPython编程\n\n自动化测试\n\nwebdriver\n\nselenium\n\nro...,Python,[],669,121,0,0,[CLS] [UNK] [UNK] [UNK] [UNK] - [UNK] [UNK] [UNK] 成 [UNK] [UNK] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ...
138194,adrienhahahah/ProjectMagenta,A fork of manuelcortez/TWBlue with enhansements.,"# Project Magenta\n\n## What is it?\n\nProject Magenta is a fork of [TWBlue](http://twblue.es) with enhancements. TWBlue changes are frequently merged in, and bug fixes are occasionally sent upstr...",Python,[],505,0,0,0,[CLS] a fork of manuel ##cor ##te ##z / t ##w ##bl ##ue with en ##han ##se #

In [81]:
def find_original_repo(rows):

    """This function checks to see whether the context tokens show up in the original repo description,
    and if so, returns metadata about the the slug, description, topic, stars of that repo"""

    text = rows['tokens'].replace('**', '')
    text = text[55:70]

    if repo_data['tokens'].str.contains(text, regex=False).any() == True :
        row = repo_data[repo_data['tokens'].str.contains(text, regex=False)].values[0]
        slug, description, topic, stars = row[0], row[1], row[7], row[6]
        return slug, description, topic, stars
    else:
        return None, None, None, None

In [98]:
#List of keywords that you want to compare
keywords = ['python', "r"]

# How to color the points in the plot. The other option is "period" for time period
color_by = 'word'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
df[['slug', 'description', 'topic', 'stars']] = df.apply(find_original_repo, axis='columns', result_type='expand')


In [88]:
df

,x,y,context,tokens,word,slug,description,topic,stars
0,-0.077461,0.031010,"a package for representing electrophysiology data in **python**, together with support for reading a wide range of","a package for representing electro ##phy ##sio ##logy data in python , together with support for reading a wide range of",python,NeuralEnsemble/python-neo,"Neo is a package for representing electrophysiology data in Python, together with support for reading a wide range of neurophysiology file formats",187.0,178.0
1,-0.071741,0.008274,. distributed systems emulator--old repository a **python** api for fragalysis ( fragalysis.,. distributed systems em ##ulator - - old repository a python api for fra ##gal ##ysis ( fra ##gal ##ysis .,python,xchem/fragalysis-api,A python API for fragalysis (fragalysis.diamond.ac.uk),9.0,0.0
2,-0.080251,0.029016,. streaming processing system designed like apache storm architecture the **python** sdk for the rackspace cloud braintree drop,. streaming processing system designed like apache storm architecture the python sd ##k for the racks ##pace cloud brain ##tree drop,python,Colearo/Crane,Streaming Processing System designed like Apache Storm architecture,0.0,0.0
3,-0.045493,-0.186065,book cgm 主 [UNK] a web interface for the rpi cam warning: this code was merged into node,##book c ##gm 主 [UNK] a web interface for the r ##pi cam warning : this code was merged into node,r,None,None,NaN,NaN
4,-0.079520,0.039064,"p2p framework an apache2 licensed, unofficial **python** client for the mygeotab api. linux","p ##2 ##p framework an apache ##2 licensed , unofficial python client for the my ##ge ##ota ##b api . linux",python,Geotab/mygeotab-python,"An Apache2 Licensed, unofficial Python client for the MyGeotab API.",30.0,10.0
...,...,...,...,...,...,...,...,...,...
186,-0.068503,0.007431,swift binary protocol client libraries a retro game engine for **python** a wordpress plugin to parse gutenberg,swift binary protocol client libraries a retro game engine for python a word ##press plug ##in to par ##se gut ##enberg,python,kitao/pyxel,A retro game engine for Python,8277.0,527.0
187,-0.079340,0.035560,a powerful spider ( web crawler ) system in **python**. a scratch pad repository to track efforts to produce,a powerful spider ( web crawl ##er ) system in python . a scratch pad repository to track efforts to produce,python,chcosta/source-build,A scratch pad repository to track efforts to produce a source tarball of the .NET Core CLI and all its components,0.0,0.0
188,-0.071580,0.008123,qp data; pytrx is a **python** object-oriented programme created for the purpose of calculating,##q ##p data ; p ##yt ##r ##x is a python object - oriented programme created for the purpose of calculating,python,PennyHow/PyTrx,PyTrx is a Python object-oriented programme created for the purpose of calculating real-world measurements from oblique images and time-lapse image series. Its primary purpose is to obtain velocit...,19.0,12.0
189,-0.076593,0.038672,library a c + + library ( bindings for **python** and c# ) for interfacing with the,library a c + + library ( binding ##s for python and c # ) for inter ##fa ##cing with the,python,analogdevicesinc/libm2k,A C++ library (bindings for Python and C#) for interfacing with the ADALM2000,16.0,18.0


In [94]:

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
#df = df.rename(columns={'slug': 'repo_slug', 'context': 'description'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: Python, R").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    #href="link",
    tooltip=['slug', 'description', 'topic', 'stars']
    ).interactive().properties(
    width=500,
    height=500
)

alt.Chart(...)